# Notebook 01: Data Preparation

This notebook handles:
1. Loading raw data files (enrolment, demographic updates, biometric updates)
2. Validating schemas and checking data quality
3. Merging datasets on (date, state, district)
4. Calculating core derived metrics
5. Saving processed data to parquet format

In [ ]:
# Cell 1: Imports and Setup
import sys
from pathlib import Path

# Add src to path for imports
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from src import config
from src.preprocessing import (
    load_enrolment,
    load_demographic,
    load_biometric,
    merge_datasets,
    calculate_core_metrics,
    handle_missing_values,
    save_processed,
    validate_schema,
    check_date_continuity
)

# Display settings
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8-whitegrid')

print("✓ Imports loaded successfully")

## 1. Load Raw Data

In [ ]:
# Cell 2: Load Enrolment Data
print("Loading enrolment data...")
enrol_df = load_enrolment(config.ENROLMENT_FILE)

print(f"\nEnrolment Dataset Summary:")
print(f"  - Rows: {len(enrol_df):,}")
print(f"  - Columns: {enrol_df.columns.tolist()}")
print(f"  - Date range: {enrol_df['date'].min()} to {enrol_df['date'].max()}")
print(f"  - States: {enrol_df['state'].nunique()}")
print(f"  - Districts: {enrol_df['district'].nunique()}")
print(f"  - Total enrolments: {enrol_df[config.METRIC_ENROLMENT_TOTAL].sum():,.0f}")

enrol_df.head()

In [ ]:
# Cell 3: Load Demographic Update Data
print("Loading demographic update data...")
demo_df = load_demographic(config.DEMOGRAPHIC_UPDATE_FILE)

print(f"\nDemographic Update Dataset Summary:")
print(f"  - Rows: {len(demo_df):,}")
print(f"  - Columns: {demo_df.columns.tolist()}")
print(f"  - Date range: {demo_df['date'].min()} to {demo_df['date'].max()}")
print(f"  - States: {demo_df['state'].nunique()}")
print(f"  - Districts: {demo_df['district'].nunique()}")
print(f"  - Total updates: {demo_df[config.METRIC_DEMOGRAPHIC_UPDATES_TOTAL].sum():,.0f}")

demo_df.head()

In [ ]:
# Cell 4: Load Biometric Update Data
print("Loading biometric update data...")
bio_df = load_biometric(config.BIOMETRIC_UPDATE_FILE)

print(f"\nBiometric Update Dataset Summary:")
print(f"  - Rows: {len(bio_df):,}")
print(f"  - Columns: {bio_df.columns.tolist()}")
print(f"  - Date range: {bio_df['date'].min()} to {bio_df['date'].max()}")
print(f"  - States: {bio_df['state'].nunique()}")
print(f"  - Districts: {bio_df['district'].nunique()}")
print(f"  - Total updates: {bio_df[config.METRIC_BIOMETRIC_UPDATES_TOTAL].sum():,.0f}")

bio_df.head()

## 2. Data Quality Checks

In [ ]:
# Cell 5: Schema Validation
print("=" * 60)
print("SCHEMA VALIDATION RESULTS")
print("=" * 60)

# Validate enrolment schema
enrol_valid, enrol_errors = validate_schema(enrol_df, config.ENROLMENT_SCHEMA, 'enrolment')
print(f"\nEnrolment: {'✓ PASSED' if enrol_valid else '✗ FAILED'}")
if not enrol_valid:
    for err in enrol_errors:
        print(f"  - {err}")

# Validate demographic schema
demo_valid, demo_errors = validate_schema(demo_df, config.DEMOGRAPHIC_SCHEMA, 'demographic')
print(f"\nDemographic: {'✓ PASSED' if demo_valid else '✗ FAILED'}")
if not demo_valid:
    for err in demo_errors:
        print(f"  - {err}")

# Validate biometric schema
bio_valid, bio_errors = validate_schema(bio_df, config.BIOMETRIC_SCHEMA, 'biometric')
print(f"\nBiometric: {'✓ PASSED' if bio_valid else '✗ FAILED'}")
if not bio_valid:
    for err in bio_errors:
        print(f"  - {err}")

In [ ]:
# Cell 6: Check Date Continuity
print("=" * 60)
print("DATE CONTINUITY CHECK")
print("=" * 60)

for name, df in [('Enrolment', enrol_df), ('Demographic', demo_df), ('Biometric', bio_df)]:
    is_cont, gaps = check_date_continuity(df, group_cols=['state', 'district'])
    print(f"\n{name}: {'✓ No significant gaps' if is_cont else f'✗ Found {len(gaps)} gaps'}")
    if not is_cont and len(gaps) > 0:
        print(gaps.head(10))

In [ ]:
# Cell 7: Missing Values Analysis
print("=" * 60)
print("MISSING VALUES ANALYSIS")
print("=" * 60)

for name, df in [('Enrolment', enrol_df), ('Demographic', demo_df), ('Biometric', bio_df)]:
    print(f"\n{name}:")
    missing = df.isnull().sum()
    missing_pct = (missing / len(df) * 100).round(2)
    missing_df = pd.DataFrame({'Missing': missing, 'Percent': missing_pct})
    print(missing_df[missing_df['Missing'] > 0] if missing_df['Missing'].sum() > 0 else "  No missing values")

## 3. Merge Datasets

In [ ]:
# Cell 8: Merge all datasets
print("Merging datasets on (date, state, district)...")
merged_df = merge_datasets(enrol_df, demo_df, bio_df)

print(f"\nMerged Dataset Summary:")
print(f"  - Rows: {len(merged_df):,}")
print(f"  - Columns: {len(merged_df.columns)}")
print(f"  - States: {merged_df['state'].nunique()}")
print(f"  - Districts: {merged_df['district'].nunique()}")
print(f"  - Date range: {merged_df['date'].min()} to {merged_df['date'].max()}")

print(f"\nColumn list:")
print(merged_df.columns.tolist())

In [ ]:
# Cell 9: Verify merge integrity
print("=" * 60)
print("MERGE INTEGRITY CHECK")
print("=" * 60)

# Check for rows with only enrolment data
enrol_only = merged_df[config.METRIC_ENROLMENT_TOTAL].notna() & merged_df[config.METRIC_DEMOGRAPHIC_UPDATES_TOTAL].isna()
demo_only = merged_df[config.METRIC_DEMOGRAPHIC_UPDATES_TOTAL].notna() & merged_df[config.METRIC_ENROLMENT_TOTAL].isna()
bio_only = merged_df[config.METRIC_BIOMETRIC_UPDATES_TOTAL].notna() & merged_df[config.METRIC_ENROLMENT_TOTAL].isna()

print(f"Rows with enrolment data only: {enrol_only.sum():,}")
print(f"Rows with demographic data only: {demo_only.sum():,}")
print(f"Rows with biometric data only: {bio_only.sum():,}")
print(f"Rows with complete data: {(~enrol_only & ~demo_only & ~bio_only).sum():,}")

## 4. Calculate Core Metrics

In [ ]:
# Cell 10: Calculate all derived metrics
print("Calculating core metrics...")
df = calculate_core_metrics(merged_df)

print(f"\nNew columns added:")
new_cols = [c for c in df.columns if c not in merged_df.columns]
for col in new_cols:
    print(f"  - {col}")

print(f"\nMetric Statistics:")
metric_cols = [
    config.METRIC_ENROLMENT_TOTAL,
    config.METRIC_TOTAL_UPDATES,
    config.METRIC_UPDATE_TO_ENROLMENT_RATIO,
    config.METRIC_ENROLMENT_VELOCITY,
    config.METRIC_ENROLMENT_VOLATILITY
]
df[metric_cols].describe()

In [ ]:
# Cell 11: Verify metric calculations
print("=" * 60)
print("METRIC VERIFICATION")
print("=" * 60)

# Verify total_updates = demographic + biometric
expected_total = df[config.METRIC_DEMOGRAPHIC_UPDATES_TOTAL].fillna(0) + df[config.METRIC_BIOMETRIC_UPDATES_TOTAL].fillna(0)
actual_total = df[config.METRIC_TOTAL_UPDATES]
match = (expected_total == actual_total).all()
print(f"total_updates calculation: {'✓ CORRECT' if match else '✗ MISMATCH'}")

# Verify update_to_enrolment_ratio
sample_idx = df[config.METRIC_ENROLMENT_TOTAL] > 0
expected_ratio = df.loc[sample_idx, config.METRIC_TOTAL_UPDATES] / df.loc[sample_idx, config.METRIC_ENROLMENT_TOTAL]
actual_ratio = df.loc[sample_idx, config.METRIC_UPDATE_TO_ENROLMENT_RATIO]
ratio_close = np.allclose(expected_ratio, actual_ratio, rtol=1e-5)
print(f"update_to_enrolment_ratio calculation: {'✓ CORRECT' if ratio_close else '✗ MISMATCH'}")

# Check velocity and volatility have been calculated
print(f"enrolment_velocity calculated: {'✓' if df[config.METRIC_ENROLMENT_VELOCITY].notna().sum() > 0 else '✗'}")
print(f"enrolment_volatility calculated: {'✓' if df[config.METRIC_ENROLMENT_VOLATILITY].notna().sum() > 0 else '✗'}")

## 5. Handle Missing Values

In [ ]:
# Cell 12: Handle missing values
print(f"Before cleaning: {len(df):,} rows")
print(f"Missing values: {df.isnull().sum().sum():,}")

df = handle_missing_values(df)

print(f"\nAfter cleaning: {len(df):,} rows")
print(f"Missing values: {df.isnull().sum().sum():,}")

## 6. Save Processed Data

In [ ]:
# Cell 13: Save processed data
output_path = save_processed(df, config.MERGED_DATA_FILE)

print(f"\n" + "=" * 60)
print("DATA PREPARATION COMPLETE")
print("=" * 60)
print(f"✓ Processed {len(df):,} rows across {df['district'].nunique()} districts")
print(f"✓ Saved to: {output_path}")
print(f"\nFile size: {output_path.stat().st_size / 1024 / 1024:.2f} MB")

In [ ]:
# Cell 14: Final dataset preview
print("Final Dataset Sample:")
df.head(10)

In [ ]:
# Cell 15: Dataset info
print("Dataset Info:")
df.info()